In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
import warnings
warnings.filterwarnings('ignore')

# Data Import

In [2]:
data_1 = pd.read_csv('./data/2019.csv', index_col=None, encoding = 'gb18030')
data_2 = pd.read_csv('./data/2013.08-2018.12.csv', index_col=None, encoding = 'gb18030')
data_3 = pd.read_csv('./data/2010.01-2013.08.csv', index_col=None, encoding = 'gb18030')
data_4 = pd.read_csv('./data/2019.01-2019.04.csv', index_col=None, encoding = 'gb18030')
data_1 = data_1[data_1['订单创建时间']>'2019-04-30 23:59:59']

In [3]:
print(f'The shape of data1 is {data_1.shape}, shape of data2 is {data_2.shape}, shape of data3 is {data_3.shape},shape of data4 is {data_4.shape}')

The shape of data1 is (2618, 61), shape of data2 is (92256, 30), shape of data3 is (45625, 30),shape of data4 is (4226, 30)


In [4]:
data = pd.concat([data_1.loc[:,data_2.columns], data_2, data_3,data_4.loc[:,data_2.columns]], ignore_index=True)
data = data.rename(columns={'订单编号':'OrderID', '买家会员名':'MemberName', '买家支付宝账号':'AlipayAccount',
                            '买家应付货款':'OwnedAmount', '买家应付邮费':'DeliveryFee', '买家支付积分':'PointsPay',
                            '总金额':'Total', '返点积分':'PointsEarned', '买家实际支付金额':'DirectPay' ,
                            '买家实际支付积分':'PointsPay', '订单状态':'OrderStatus', '买家留言':'BuyerMessage',
                            '收货人姓名':'ReceiverName', '收货地址 ':'ReceivingAdd', '运送方式':'DeliverMethod',
                            '联系电话 ':'ContactPhone', '联系手机':'CellPhone', '订单创建时间':'OrderTime',
                            '订单付款时间 ':'PayTime', '宝贝标题 ':'ItemName', '宝贝种类 ':'ItemCate',
                            '物流单号 ':'DeliverNo', '物流公司':'DeliverCompany', '订单备注':'OrderNote', 
                            '宝贝总数量':'OrderItemNo', '店铺Id':'StoreID', '店铺名称':'StoreName', 
                            '确认收货时间':'DeliveredTime', '打款商家金额':'StoreReceived', '是否村淘订单':'CunTaoOrder'
                            })

In [5]:
data = data.drop(data.loc[(data['OrderTime']>'2019-08-01 00:00:00') |\
                          (data['OrderStatus'] == '卖家已发货，等待买家确认')].index,axis=0)

In [6]:
data.shape

(144269, 30)

# Data Clean

- There are total 13 columns with missing data:  
        AlipayAccount, BuyerMessage, ReceiverName, DeliverMethod, ContactPhone,  
        CellPhone, PaytTime, DeliverNo, DeliverCompany, OrderNote,  
        StoreID, StoreName, DeliveredTime  

In [7]:
np.sum(data.isnull(),axis=0)

OrderID                0
MemberName             0
AlipayAccount       1077
OwnedAmount            0
DeliveryFee            0
PointsPay              0
Total                  0
PointsEarned           0
DirectPay              0
PointsPay              0
OrderStatus            0
BuyerMessage      128876
ReceiverName           1
ReceivingAdd           0
DeliverMethod          1
ContactPhone      117186
CellPhone            962
OrderTime              0
PayTime            27827
ItemName             398
ItemCate               0
DeliverNo          32979
DeliverCompany     32979
OrderNote         125565
OrderItemNo            0
StoreID           141178
StoreName          54394
DeliveredTime      33036
StoreReceived          0
CunTaoOrder            0
dtype: int64

In [8]:
# Create Province & City
data.loc[156, 'ReceivingAdd'] = '海南省 文昌市 文城镇 海南省文昌市文昌中学'
data.loc[86947, 'ReceivingAdd'] = '四川省 成都市 天府新区 华阳街道音乐广场贝康宠物医院(610213)'
data['Province'] = list(map(lambda s: s.split(' ')[0], data['ReceivingAdd']))
data['City'] = list(map(lambda s: s.split(' ')[1], data['ReceivingAdd']))

- There is no record missing CellPhone, ContactPhone, AlipayAccount at the same time
 - CellPhone 962 missing values, fill with ContactPhone
 - AlipayAccount 1601 missing values, fill with CellPhone, ContactPhone

In [9]:
data['CellPhone'] = data['CellPhone'].fillna(data['ContactPhone'])
data['AlipayAccount'] = data['AlipayAccount'].fillna(data['CellPhone'])

- Missing BuyerMessage and OrderNote means customers left no message
 - BuyerMessage 124885 missing values, fill with 'No'
 - OrderNote 22259 missing values, fill with 'No'

In [10]:
data['BuyerMessage'] = data['BuyerMessage'].fillna('No')
data['OrderNote'] = data['OrderNote'].fillna('No')
data['OrderNote'] = data['OrderNote'].apply(lambda s: 'No' if s=="'null" else s)

- There is one record with no information, delete it

In [11]:
data[data['ReceiverName'].isna()] 

,OrderID,MemberName,AlipayAccount,OwnedAmount,DeliveryFee,PointsPay,Total,PointsEarned,DirectPay,PointsPay,OrderStatus,BuyerMessage,ReceiverName,ReceivingAdd,DeliverMethod,ContactPhone,CellPhone,OrderTime,PayTime,ItemName,ItemCate,DeliverNo,DeliverCompany,OrderNote,OrderItemNo,StoreID,StoreName,DeliveredTime,StoreReceived,CunTaoOrder,Province,City
73583,1053302034007093,clearcatnn,clearcatnn@hotmail.com,165.0,0.0,0.0,165.0,82.0,165.0,0.0,交易成功,No,NaN,,NaN,NaN,NaN,2015-05-17 16:02:14,2015-05-17 16:02:34,科博磁力棒磁铁儿童益智玩具男女孩智力拼装玩具百变早教磁力积木,1.0,NaN,NaN,No,1.0,NaN,母婴惠童专营店,2015-05-22 18:42:27,165.00元,否,,
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,海南省 文昌市 文城镇 海南省文昌市文昌中学,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,海南省,文昌市


In [12]:
data = data[~data['ReceiverName'].isna()]

- Records with missing PaytTime and DeliveredTime are all from closed orders, which means the customers closed the orders before paying.
 - PaytTime 26640 missing values(all included in closed orders), fill with 0
 - DeliveredTime 31516 missing values(all included in closed orders), fill with 0

In [13]:
t1 = data[(data['PayTime'].isnull()) & (data['OrderStatus'] == '交易成功')].shape[0]
t2 = data[(data['DeliveredTime'].isnull()) & (data['OrderStatus'] == '交易成功')].shape[0]
print(f'The number of success order without PaytTime is {t1}, the number of success order without DeliveredTime is {t2}')

The number of success order without PaytTime is 0, the number of success order without DeliveredTime is 0


In [14]:
data['PayTime'] = data['PayTime'].fillna(0)
data['DeliveredTime'] = data['DeliveredTime'].fillna(0)

- ContactPhone, StoreID, StoreName, DeliverNo are useless columns, delete

In [15]:
data = data.drop(columns = (['ContactPhone', 'StoreID','StoreName', 'DeliverNo']), axis = 0)
data = data.drop(data.loc[(data['OrderTime']>'2019-08-01 00:00:00') | (data['OrderStatus'] == '卖家已发货，等待买家确认')].index,axis=0)

- Check the ItemName and OrderNote, delete records which just pay for delivery fee.
- Impute DeliverCompany with mode in every province
- After imputation, the left 29693 missing in DeliverCompany are from closed orders, fill with No

In [16]:
data = data[~(data['ItemName'] == '邮费运费链接补差价专拍')]

In [17]:
def delete_fake_orders(df, key_words):
    for word in key_words:
        tmp = df[(df['DeliverCompany'].isnull()) & (df['OrderStatus'] == '交易成功')]
        df = df.drop(tmp[tmp['OrderNote'].apply(lambda x : re.search(word, x) != None)].index, axis = 0)
    return df

def impute_deliver_company(df):
    impute_index = df[(df['DeliverCompany'].isnull())&(df['OrderStatus'] == '交易成功')].index
    for index in impute_index:
        df.loc[index, 'DeliverCompany'] = df[(~df['DeliverCompany'].isnull())&(df['Province'] == df.loc[index].Province)].DeliverCompany.mode()[0]
    return df

In [18]:
data = delete_fake_orders(data, ['邮','费','差','价','运','费','补','重复','不要','不发','关闭'])
data = impute_deliver_company(data)
data['DeliverCompany'] = data['DeliverCompany'].fillna('No')
data['ItemName'] = data['ItemName'].fillna('No')

In [19]:
data.isnull().values.any()

False

**Create time features**

In [20]:
data['OrderTime'] = pd.to_datetime(data['OrderTime'])
data['PayTime'] = pd.to_datetime(data['PayTime'])
data['DeliveredTime'] = pd.to_datetime(data['DeliveredTime'])

In [21]:
# create new col: time window between PaytTime & OrderTime
data['OrderToPay(s)'] = (data['PayTime'] - data['OrderTime']).astype('timedelta64[s]')
# create new col: time 
data['PayToReceive(h)'] = (data['DeliveredTime'] - data['PayTime']).astype('timedelta64[h]')

In [22]:
data.head(3)

,OrderID,MemberName,AlipayAccount,OwnedAmount,DeliveryFee,PointsPay,Total,PointsEarned,DirectPay,PointsPay,OrderStatus,BuyerMessage,ReceiverName,ReceivingAdd,DeliverMethod,CellPhone,OrderTime,PayTime,ItemName,ItemCate,DeliverCompany,OrderNote,OrderItemNo,DeliveredTime,StoreReceived,CunTaoOrder,Province,City,OrderToPay(s),PayToReceive(h)
454,"=""561099105503109255""",雷霆3战机,342239175@qq.com,154.84,0.0,0.0,154.84,77.0,154.84,0.0,交易成功,No,陈丹,福建省 福州市 仓山区 东升街道东园路10号金辉莱茵城14号楼（东辉社区）(000000),快递,'13405907147,2019-07-31 23:28:46,2019-07-31 23:28:58,迪宝乐电子积木儿童益智拼装电路玩具男女孩物理教科书5-6-12岁,1.0,韵达快递,No,1.0,2019-08-09 22:38:05,154.84元,否,福建省,福州市,12.0,215.0
455,"=""561103267468252376""",konglingsheng888,kls666@sina.com,109.00,0.0,0.0,109.00,54.0,109.00,0.0,交易成功,No,孔令晟,安徽省 铜陵市 铜官山区 铜官山区虚镇秀水山庄4栋401(244000),快递,'13856271285,2019-07-31 22:58:03,2019-07-31 22:58:30,迪士尼书包小学生男1-3-4年级汽车麦昆卡通儿童护脊双肩背包,1.0,圆通速递,No,1.0,2019-08-03 19:47:38,109.00元,否,安徽省,铜陵市,27.0,68.0
456,"=""561084450761550335""",更好的明天1992,18067226706,103.00,0.0,0.0,103.00,51.0,103.00,0.0,交易成功,No,吴文洁,河南省 商丘市 睢县 城关镇西门里赵家胡同(000000),快递,'18067222105,2019-07-31 22:55:06,2019-07-31 22:55:41,迪宝乐电子积木儿童益智拼装电路玩具男女孩物理教科书5-6-12岁,1.0,韵达快递,No,1.0,2019-08-11 08:32:16,103.00元,否,河南省,商丘市,35.0,249.0


Notes from 2019.csv: 
1. 订单编号 is unique   ???
2. '总金额' = '买家应付货款' + '买家应付邮费', (32 records with 买家应付邮费)
3. '买家实际支付金额' =  '总金额' -  '退款金额'   
4. '返点积分' = 0.5 * '买家实际支付金额' + '买家支付积分'
5. '打款商家金额' = '买家实际支付金额'

In [23]:
data[data['OrderID'] == 403137289407813]

,OrderID,MemberName,AlipayAccount,OwnedAmount,DeliveryFee,PointsPay,Total,PointsEarned,DirectPay,PointsPay,OrderStatus,BuyerMessage,ReceiverName,ReceivingAdd,DeliverMethod,CellPhone,OrderTime,PayTime,ItemName,ItemCate,DeliverCompany,OrderNote,OrderItemNo,DeliveredTime,StoreReceived,CunTaoOrder,Province,City,OrderToPay(s),PayToReceive(h)
94873,403137289407813,天涯海角516087,13584860290,78.0,0.0,0.0,78.0,39.0,78.0,0.0,交易成功,No,胡国艳,江苏省 苏州市 园区 唯亭镇青苑四区14栋2单元(215121),快递,'13782986509,2013-08-19 22:29:27,2013-08-19 22:31:20,和乐虎早教机故事机幼儿和乐熊和乐族儿童益智玩具会说话的巧虎,1.0,申通E物流,No,1.0,2013-08-21 19:00:32,78.00元,否,江苏省,苏州市,113.0,44.0
94874,403137289407813,天涯海角516087,13584860290,78.0,0.0,0.0,78.0,39.0,78.0,0.0,交易成功,No,胡国艳,江苏省 苏州市 园区 唯亭镇青苑四区14栋2单元(215121),快递,'13782986509,2013-08-19 22:29:27,2013-08-19 22:31:20,和乐虎早教机故事机幼儿和乐熊和乐族儿童益智玩具会说话的巧虎,1.0,申通E物流,No,1.0,2013-08-21 19:00:32,78.00元,否,江苏省,苏州市,113.0,44.0


In [24]:
data = data.drop_duplicates()

In [26]:
data.shape

(143939, 30)

In [29]:
data.to_csv('./data/after_clean.csv', index = False, encoding = 'gb18030')

# Analysis -Maomao

In [ ]:
excellent_payment_amount = [[2017,8,300766],[2017,9,268919],[2017,10,231100],[2017,11,381938],[2017,12,323107],
                  [2018,1,314732],[2018,2,182211],[2018,3,296708],[2018,4,259451],[2018,5,364607],
                  [2018,6,355321],[2018,7,357994],[2018,8,409112],[2018,9,361518],[2018,10,344064],[2018,11,498062],
                  [2018,12,460035],[2019,1,395456],[2019,2,299989],[2019,3,365981],[2019,4,335556],
                  [2019,5,418450],[2019,6,316545]]

average_payment_amount = [[2017,8,213675],[2017,9,180691],[2017,10,155361],[2017,11,256130],[2017,12,227572],
                   [2018,1,209429],[2018,2,120317],[2018,3,197909],[2018,4,186440],[2018,5,244298],
                   [2018,6,232010],[2018,7,250811],[2018,8,281516],[2018,9,256647],[2018,10,234571],
                   [2018,11,344893],[2018,12,320763],[2019,1,268417],[2019,2,202224],[2019,3,251131],
                   [2019,4,233499],[2019,5,272901],[2019,6,210169]]

excellent_view_count = [[2017,8,173931],[2017,9,138438],[2017,10,134606],[2017,11,193768],[2017,12,154619],
                  [2018,1,157413],[2018,2,120345],[2018,3,148101],[2018,4,125594],[2018,5,141412],
                  [2018,6,142020],[2018,7,164316],[2018,8,187370],[2018,9,154759],[2018,10,175340],
                  [2018,11,212596],[2018,12,192076],[2019,1,183337],[2019,2,179527],
                  [2019,3,178862],[2019,4,177142],[2019,5,182135],[2019,6,157187]]

average_view_count = [[2017,8,100230],[2017,9,74885],[2017,10,72665],[2017,11,98999],[2017,12,76362],
               [2018,1,80478],[2018,2,57998],[2018,3,74096],[2018,4,67236],[2018,5,73419],
                [2018,6,75429],[2018,7,87338],[2018,8,99806],[2018,9,82116],[2018,10,90074],
                [2018,11,112136],[2018,12,95180],[2019,1,86973],[2019,2,86342],
                [2019,3,91134],[2019,4,79997],[2019,5,89195],[2019,6,74565]]

excellent_conversion_rate = [[2017,8,0.054],[2017,9,0.0554],[2017,10,0.0524],[2017,11,0.056],[2017,12,0.0579],
                           [2018,1,0.0575],[2018,2,0.047],[2018,3,0.0619],[2018,4,0.062],[2018,5,0.0682],
                           [2018,6,0.0673],[2018,7,0.0653],[2018,8,0.0655],[2018,9,0.069],[2018,10,0.0675],
                          [2018,11,0.0742],[2018,12,0.0722],[2019,1,0.0668],[2019,2,0.0624],[2019,3,0.0688],
                          [2019,4,0.0705],[2019,5,0.071],[2019,6,0.0697]],
average_conversion_rate = [[2017,8,0.0265],[2017,9,0.0295],[2017,10,0.0271],[2017,11,0.031],[2017,12,0.0312],
                         [2018,1,0.0316],[2018,2,0.0243],[2018,3,0.0339],[2018,4,0.0336],[2018,5,0.039],
                         [2018,6,0.036],[2018,7,0.034],[2018,8,0.0344],[2018,9,0.0366],[2018,10,0.034],
                         [2018,11,0.0401],[2018,12,0.0396],[2019,1,0.0362],[2019,2,0.0309],[2019,3,0.0364],
                          [2019,4,0.0358],[2019,5,0.0382],[2019,6,0.0372]]

excellent_advertising_fee = [[2017,8,35899],[2017,9,30498],[2017,10,31910],[2017,11,39351],[2017,12,34908],
                            [2018,1,33828],[2018,2,19234],[2018,3,31986],[2018,4,33536],[2018,5,43895],
                            [2018,6,43778],[2018,7,43512],[2018,8,43364],[2018,9,39295],[2018,10,42594],
                            [2018,11,54724],[2018,12,46435],[2019,1,42145],[2019,2,30593],[2019,3,40873],
                            [2019,4,40557],[2019,5,42032],[2019,6,34852]]

average_advertising_fee = [[2017,8,14153],[2017,9,13095],[2017,10,12926],[2017,11,15514],[2017,12,14385],
                            [2018,1,14421],[2018,2,7281],[2018,3,13557],[2018,4,14237],[2018,5,17891],
                            [2018,6,18090],[2018,7,18464],[2018,8,18132],[2018,9,16464],[2018,10,17299],
                            [2018,11,19206],[2018,12,17596],[2019,1,16065],[2019,2,11749],[2019,3,16344],
                            [2019,4,15594],[2019,5,16994],[2019,6,14016]]

excellent_payment_amount = pd.DataFrame(excellent_payment_amount)
excellent_payment_amount.columns = ["year","month","value"]

average_payment_amount = pd.DataFrame(average_payment_amount)
average_payment_amount.columns = ["year","month","value"]

excellent_view_count = pd.DataFrame(excellent_view_count)
excellent_view_count.columns = ["year","month","value"]

average_view_count = pd.DataFrame(average_view_count)
average_view_count.columns = ["year","month","value"]

excellent_conversion_rate = pd.DataFrame(excellent_conversion_rate)
excellent_conversion_rate.columns = ["year","month","value"]

average_conversion_rate = pd.DataFrame(average_conversion_rate)
average_conversion_rate.columns = ["year","month","value"]

excellent_advertising_fee = pd.DataFrame(excellent_advertising_fee)
excellent_advertising_fee.columns = ["year","month","value"]

average_advertising_fee = pd.DataFrame(average_advertising_fee)
average_advertising_fee.columns = ["year","month","value"]

In [ ]:
data['year'] = data['PayTime'].apply(lambda x: x.year if x.year >2000 else '')
data['month'] = data['PayTime'].apply(lambda x: x.month if x.year >2000 else '')

In [ ]:
def f(df):
    df['year'] = df['year'].apply(lambda x: str(x))
    df['month'] = df['month'].apply(lambda x: str(x))
    def f(x):
        return (x['year']+'-'+x['month'])
    df['year_month'] = df.apply(f,axis=1)
    return df

data = f(data)

In [ ]:
data['year'] = data['year'].apply(lambda x: int(x) if x !='' else 0 )
data['month'] = data['month'].apply(lambda x: int(x) if x !='' else 0)
sum_2017_month = pd.DataFrame(data[data['year'] > 2016].groupby(data['year_month']).sum())

In [ ]:
data.reset_index(inplace = True,drop= True)

In [ ]:
sum_2017_month